In [2]:
import numpy as np
import os
import json

In [13]:
path_src = '/hdd/sdfstudio/data/colmap_sdfstudio/stump_raw/transforms.json'
rotation = '/hdd/sdfstudio/data/colmap_sdfstudio/stump_raw/rotation_aligned.npy'
path_tgt = '/hdd/sdfstudio/data/colmap_sdfstudio/stump/transforms.json'

with open(path_src, 'r') as f:
    data = json.load(f)
rotation = np.load(rotation)

In [14]:
# Camera Intrinsics
s = 4
w_new = int(np.floor(data['w'] / s))
h_new = int(np.floor(data['h'] / s))
sx = w_new / data['w']
sy = h_new / data['h']

print('sx:', sx)
print('sy:', sy)

data['w'] = w_new
data['h'] = h_new

data['fl_x'] *= sx
data['fl_y'] *= sy
data['cx'] *= sx
data['cy'] *= sy

print('w:', data['w'])
print('h:', data['h'])
print('fl_x:', data['fl_x'])
print('fl_y:', data['fl_y'])
print('cx:', data['cx'])
print('cy:', data['cy'])


sx: 0.24989955805544395
sy: 0.25
w: 1244
h: 825
fl_x: 1131.2309502183105
fl_y: 1131.7441525093413
cx: 619.3225331448746
cy: 415.04261395491966


In [15]:
# Camera Extrinsics
frames = data['frames']
c2ws = np.array([frames[i]['transform_matrix'] for i in range(len(frames))])


ts = c2ws[:, :3, 3]
t_max = np.max(ts, axis=0)
t_min = np.min(ts, axis=0)
t_center = (t_max + t_min) / 2
print('Max:', t_max)
print('Min:', t_min)
print('Center:', t_center)
scale = np.max(t_max - t_min) / 2
print('Scale:', scale)

ts_normalized = (ts - t_center) / scale
c2ws[:, :3, 3] = ts_normalized
R_align = np.eye(4)
R_align[:3, :3] = rotation
c2ws = np.matmul(R_align, c2ws)
for i in range(len(frames)):
    frames[i]['transform_matrix'] = c2ws[i].tolist()

data['frames'] = frames
with open(path_tgt, 'w') as f:
    json.dump(data, f, indent=4)
print('Save to ', path_tgt)

Max: [4.04742559 4.09306729 3.19949501]
Min: [-3.2005408  -3.74665197 -3.99109831]
Center: [ 0.42344239  0.17320766 -0.39580165]
Scale: 3.9198596296652948
Save to  /hdd/sdfstudio/data/colmap_sdfstudio/stump/transforms.json
